In [ ]:
# %pip install fast-langdetect wordsegment fastapi gpt4all
# Entorno tt_env

In [3]:
# INSTALACIONES PARA NUEVO AMBIENTE
# %pip install wordsegment
# %pip install transformers datasets torch pandas scikit-learn
# %pip install sentencepiece
# %pip install protobuf
# %pip install ipywidgets
# %pip install 'accelerate>=0.26.0'

# RED NEURONAL

En esta sección debido al desorden se empieza a trabajar en el modelo para sugerir el nombre de código.

In [ ]:
# import pandas as pd

# file_path = "./dataset/java_dataset.csv"
# df = pd.read_csv(file_path, encoding="latin-1")


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
    # T5Tokenizer,
    Trainer,
    TrainingArguments,
)

# 1. Cargar el dataset desde un archivo CSV
file_path = "./dataset/java_dataset.csv"
df = pd.read_csv(file_path, encoding="latin-1")

# Asegurar que las columnas clave existen
assert "code" in df.columns and "suggested_name" in df.columns, (
    "El CSV debe contener 'code' y 'suggested_name'"
)

# 2. Tokenización con CodeT5
# tokenizer = T5Tokenizer.from_pretrained("Salesforce/codet5-small")

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")


def preprocess_data(examples):
    """Función para tokenizar código y nombres sugeridos"""
    inputs = [f"Generate name: {code}" for code in examples["code"]]
    targets = examples["suggested_name"]

    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(targets, max_length=20, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Convertir DataFrame a Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Aplicar tokenización al dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# 3. Dividir en train/test
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset, test_dataset = split["train"], split["test"]

# 4. Cargar el modelo CodeT5
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")


# 5. Configurar entrenamiento
training_args = TrainingArguments(
    output_dir="./codet5-fine-tuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# 6. Entrenar el modelo
trainer.train()

# 7. Guardar el modelo
model.save_pretrained("./codet5-fine-tuned")
tokenizer.save_pretrained("./codet5-fine-tuned")

# print("¡Entrenamiento completado y modelo guardado!")

Map:   0%|          | 0/8082 [00:00<?, ? examples/s]

/home/rodrigo/miniconda3/envs/tt_env/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_9098/3050600087.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")

text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=10)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "user: {user.name}"


In [ ]:
import re

from wordsegment import load, segment

load()

In [ ]:
tipo_datos = [
    "boolean",
    "char",
    "byte",
    "short",
    "int",
    "long",
    "float",
    "double",
    "String",
    "Array",
]

In [ ]:
text = """
    package com.hmkcode;

import java.io.BufferedReader;
import java.io.FileNotFoundException;
import java.io.FileReader;
import java.io.IOException;

public class textfilereader {

	private BufferedReader buffer;
	private String currentLine = "";
	public textfilereader(){ 
		
	}
	
	public void open(String file){
		
		try {
			close();
			
			buffer = new BufferedReader(new FileReader(file));
			
		} catch (FileNotFoundException e1) {
			e1.printStackTrace();
	
		}
		
	}
	
	public void close(){
		
		try {
			if(buffer != null){
				buffer.close();
				buffer = null;
			}
		} catch (IOException e) {
			e.printStackTrace();
		}
		
	}
	
	public String readLine() throws Exception{
		if(buffer != null){
			currentLine = buffer.readLine();
			
			if(currentLine == null)
				close();
			
			return currentLine;
		}
		else
			throw new Exception("No file to read...");
	}
	
	public String getCurrent(){
		return this.currentLine;
	}
	
	public boolean isReadable(){
		return (buffer != null && this.currentLine != null);
	}
}
"""

In [ ]:
# Declaración de regex para uso general
class_starter = r"(class)\s([^\{]+)\{"
funciones_static = r"(static\s){1,1}.{0,}(){1,1}"
funciones_private = r"(private\s){1,1}.{0,}(){1,1}"

In [ ]:
class_coin = re.findall(pattern=class_starter, string=text)
class_coin

In [ ]:
# Comprobamos el nombramiento de la clase
if bool(class_coin):
    # Con una expresión regular verificamos el nombramiento de la clase y su respectivo nombre
    identificador = class_coin[0][0].strip()

    t_t = text.split("\n")
    n = len(t_t)

    # Iteramos por cada linea de código hasta que terminamos con el proceso de cambiar el nombre de la clase por la forma de cammel case
    for i in range(n):
        if identificador in t_t[i]:
            sep_l = t_t[i].split()
            n_2 = len(sep_l)
            for j in range(n_2):
                if sep_l[j] != identificador and sep_l[j] != "public":
                    sep_l[j] = separate_words(sep_l[j])
                    break
            sep_l = " ".join(sep_l)
            t_t[i] = sep_l
text = "\n".join(t_t)

# De momento solo se revisa si hay nombramiento de metodos privados
if "private" in text:
    coincidencias = re.findall(pattern=funciones_private, string=text)

    if bool(coincidencias):
        # print("segundo if")
        c = coincidencias[0][0].strip()
        t_l = text.split("\n")
        for i in range(len(t_l)):
            # print(c,line)
            if c in t_l[i]:
                # print("antes del title", t_l[i].strip())
                sep = t_l[i].split()
                # print(sep.index(c))
                for j in range(len(sep)):
                    if sep[j] != c and sep[j] not in tipo_datos:
                        sep[j] = separate_words(sep[j])
                        break
                sep = " ".join(sep)
                t_l[i] = sep
text = "\n".join(t_l)

In [ ]:
dtb_code = """
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.SQLException;
import java.sql.Statement;

public class DatabaseConnection {
    private static final String URL = "jdbc:mysql://localhost:3306/tu_base_de_datos";
    private static final String USER = "tu_usuario";
    private static final String PASSWORD = "tu_contraseña";

    private Connection connection;

    public DatabaseConnection() {
        try {
            // Cargar el driver de MySQL
            Class.forName("com.mysql.cj.jdbc.Driver");
            // Establecer la conexión
            connection = DriverManager.getConnection(URL, USER, PASSWORD);
            System.out.println("Conexión exitosa a la base de datos.");
        } catch (ClassNotFoundException | SQLException e) {
            e.printStackTrace();
        }
    }

    public void ejecutarConsulta(String consulta) {
        try (Statement statement = connection.createStatement();
             ResultSet resultSet = statement.executeQuery(consulta)) {
            while (resultSet.next()) {
                System.out.println("Resultado: " + resultSet.getString(1));
            }
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }

    public void cerrarConexion() {
        try {
            if (connection != null && !connection.isClosed()) {
                connection.close();
                System.out.println("Conexión cerrada.");
            }
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) {
        DatabaseConnection db = new DatabaseConnection();
        db.ejecutarConsulta("SELECT * FROM tu_tabla");
        db.cerrarConexion();
    }
}

"""

In [ ]:
class_coin = re.findall(pattern=class_starter, string=dtb_code)
class_coin


In [ ]:
def is_camell_case(word: str) -> bool:
    """Función para determinar si la case ya esta en formato camell case, de ser así retorna un True

    Args:
        word (str): Nombre de la clase o el método

    Returns:
        bool: Valor booleano acorde a si es cammel case o no
    """
    word = word.strip()

    sep = segment(word)

    final = "".join([item.capitalize() for item in sep])

    if final.strip() == word:
        return True
    elif final.strip() != word:
        return False

In [ ]:
is_camell_case(class_coin[0][1].strip())

In [ ]:
from gpt4all import GPT4All

llm = GPT4All(
    model_name="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
    model_path="./models",
    verbose=True,
    allow_download=False,
)

with llm.chat_session():
    print(
        llm.generate(
            f"Dime si este código de java cumple con las buenas prácticas de nombramiento del libro de clean code de Robert C. Martin: {dtb_code}, si las variables no cumplen con ser lo suficientemente descriptivas, retorna un diccionario de python donde las llaves sean los nombres de las variables actuales y el valor sea el nuevo nombre. No me expliques nada, devuelve unicamente el diccionario sin dar explicaciones ni agregar ni un comentario"
        )
    )